In [43]:
import nibabel as nib
import numpy as np
from pathlib import Path

right_path = Path(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0xZr07hZr1\manual_segmentations\nii\Mandible_R.nii.gz")
left_path = Path(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0xZr07hZr1\manual_segmentations\nii\Mandible_L.nii.gz")
out_path = Path(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0xZr07hZr1\comparison\mandible_manual.nii.gz")

right_img = nib.load(right_path)
left_img = nib.load(left_path)

right_data = right_img.get_fdata()
left_data = left_img.get_fdata()

combined_data = np.logical_or(right_data, left_data).astype(np.uint8)

combined_img = nib.Nifti1Image(combined_data, affine=right_img.affine, header=right_img.header)

nib.save(combined_img, out_path)

print(f"Combined segmentation saved to: {out_path}")

Combined segmentation saved to: Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0xZr07hZr1\comparison\mandible_manual.nii.gz


In [44]:
import SimpleITK as sitk
import numpy as np

def fill_and_balance_volume(mask_img, dilate_radius=4, max_iter=10):
    mask = sitk.Cast(mask_img > 0, sitk.sitkUInt8)
    original_vol = np.count_nonzero(sitk.GetArrayFromImage(mask))

    dilated = sitk.BinaryDilate(mask, [dilate_radius]*3)

    eroded = dilated
    prev_diff = float('inf')
    best = eroded

    for r in range(1, max_iter+1):
        test = sitk.BinaryErode(dilated, [r]*3)
        vol = np.count_nonzero(sitk.GetArrayFromImage(test))
        diff = abs(vol - original_vol)
        if diff < prev_diff:
            best, prev_diff = test, diff
        else:
            break

    return best

mandible = sitk.ReadImage(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0xZr07hZr1\comparison\mandible_totalseg.nii.gz")
filled_balanced = fill_and_balance_volume(mandible, dilate_radius=5)
sitk.WriteImage(filled_balanced, r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0xZr07hZr1\comparison\mandible_totalseg_filled.nii.gz")
print("Done")

Done


In [ ]:
#same plot overlay
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
plt.rcParams["mathtext.fontset"] = "custom"
plt.rcParams["mathtext.rm"] = "Times New Roman"
plt.rcParams["mathtext.it"] = "Times New Roman:italic"
plt.rcParams["mathtext.bf"] = "Times New Roman:bold"

ct_img = nib.load(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\CT.nii.gz")
X = ct_img.get_fdata()
X = (X - np.min(X)) / (np.max(X) - np.min(X))  # Normalize to [0, 1] (binary)

seg_paths = {
    "Manual Mandible": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\mandible_manual.nii.gz",
    "TotalSegmentator Mandible (filled holes)": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\mandible_totalseg_filled.nii.gz",
}

segments = {name: nib.load(path).get_fdata() for name, path in seg_paths.items()}
spacing = ct_img.header.get_zooms()

overlay_colors = {
    "Manual Mandible": (0.2, 1.0, 0.2, 0.35),   # blue
    "TotalSegmentator Mandible (filled holes)": (1.0, 0.2, 0.2, 0.35),
}

def show_slice(slice_index, plane, alpha=0.35, patient_id="UIDQQ0xZr07hZr1"):
    plt.figure(figsize=(6, 6))

    if plane == 'Axial':
        img_slice = np.rot90(X[:, :, slice_index])
        aspect = spacing[1] / spacing[0]
        seg_slices = {n: np.rot90(S[:, :, slice_index]) for n, S in segments.items()}

    elif plane == 'Coronal':
        img_slice = np.flipud(np.rot90(X[:, slice_index, :]))
        aspect = spacing[2] / spacing[0]
        seg_slices = {n: np.flipud(np.rot90(S[:, slice_index, :])) for n, S in segments.items()}

    elif plane == 'Sagittal':
        img_slice = np.flipud(np.rot90(X[slice_index, :, :]))
        aspect = spacing[2] / spacing[1]
        seg_slices = {n: np.flipud(np.rot90(S[slice_index, :, :])) for n, S in segments.items()}

    plt.imshow(img_slice, cmap='gray', origin='lower', aspect=aspect)

    for name, seg_slice in seg_slices.items():
        mask = seg_slice > 0.5
        if np.any(mask):
            rgba = list(overlay_colors[name])
            rgba[3] = alpha  
            overlay = np.zeros((*mask.shape, 4))
            overlay[mask] = rgba
            plt.imshow(overlay, origin='lower', aspect=aspect)

    plt.title(f"{plane} slice {slice_index}")
    plt.axis("off")


    legend_handles = [
        plt.Line2D([0], [0], color=overlay_colors[n][:3], lw=4, label=n)
        for n in overlay_colors
    ]
    leg=plt.legend(handles=legend_handles, loc='upper right', fontsize=8, frameon=False, labelcolor='white', title="Segments")
    leg.get_title().set_fontsize(10)
    leg.get_title().set_color('white')
    clear_output(wait=True)
    display(plt.gcf())
    plt.close()


plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=X.shape[2] // 2,
    min=0,
    max=X.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)


def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = X.shape[2] - 1
        slice_slider.value = X.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = X.shape[1] - 1
        slice_slider.value = X.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = X.shape[0] - 1
        slice_slider.value = X.shape[0] // 2


plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(show_slice, slice_index=slice_slider, plane=plane_dropdown, alpha=alpha_slider)


interactive(children=(IntSlider(value=120, description='Slice:', layout=Layout(width='80%'), max=239), Dropdow…

<function __main__.show_slice(slice_index, plane, alpha=0.35, patient_id='UIDQQ0xZr07hZr1')>

In [ ]:
#to view on subplots
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

ct_img = nib.load(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\CT.nii.gz")
X = ct_img.get_fdata()
X = (X - np.min(X)) / (np.max(X) - np.min(X))  # Normalize to [0, 1]
spacing = ct_img.header.get_zooms()

seg_paths = {
    "Manual Mandible": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\mandible_manual.nii.gz",
    "TotalSegmentator Mandible (filled holes)": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\comparison\mandible_totalseg_filled.nii.gz",
}
segments = {name: nib.load(path).get_fdata() for name, path in seg_paths.items()}

overlay_colors = {
    "Manual Mandible": (0.2, 1.0, 0.2, 0.35),   # blue
    "TotalSegmentator Mandible (filled holes)": (1.0, 0.2, 0.2, 0.35),
}

def show_slice(slice_index, plane, alpha=0.35):
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    plt.tight_layout()

    if plane == 'Axial':
        img_slice = np.rot90(X[:, :, slice_index])
        aspect = spacing[1] / spacing[0]
        seg_slices = {n: np.rot90(S[:, :, slice_index]) for n, S in segments.items()}

    elif plane == 'Coronal':
        img_slice = np.flipud(np.rot90(X[:, slice_index, :]))
        aspect = spacing[2] / spacing[0]
        seg_slices = {n: np.flipud(np.rot90(S[:, slice_index, :])) for n, S in segments.items()}

    elif plane == 'Sagittal':
        img_slice = np.flipud(np.rot90(X[slice_index, :, :]))
        aspect = spacing[2] / spacing[1]
        seg_slices = {n: np.flipud(np.rot90(S[slice_index, :, :])) for n, S in segments.items()}

    # left subplot 
    ax_left = axes[0]
    ax_left.imshow(img_slice, cmap='gray', origin='lower', aspect=aspect)

    for name in ["mandible_manual"]:
        if name in seg_slices:
            mask = seg_slices[name] > 0.5
            if np.any(mask):
                rgba = list(overlay_colors[name])
                rgba[3] = alpha
                overlay = np.zeros((*mask.shape, 4))
                overlay[mask] = rgba
                ax_left.imshow(overlay, origin='lower', aspect=aspect)

    ax_left.set_title(f"{plane} slice {slice_index} — Manual Segmentation")
    ax_left.axis("off")

    legend_handles_left = [
        plt.Line2D([0], [0], color=overlay_colors[n][:3], lw=4, label=n)
        for n in ["mandible_manual"]
    ]
    leg_left = ax_left.legend(
        handles=legend_handles_left,
        loc='lower right',
        fontsize=8,
        frameon=True,
        facecolor='black',
        edgecolor='white',
        labelcolor='white',
        title="Segments",
        title_fontsize=9
    )

    plt.setp(leg_left.get_texts(), color='white')
    leg_left.get_title().set_color('white')

    # right subplot
    ax_right = axes[1]
    ax_right.imshow(img_slice, cmap='gray', origin='lower', aspect=aspect)

    for name in ["mandible_totalseg_filled"]:
        if name in seg_slices:
            mask = seg_slices[name] > 0.5
            if np.any(mask):
                rgba = list(overlay_colors[name])
                rgba[3] = alpha
                overlay = np.zeros((*mask.shape, 4))
                overlay[mask] = rgba
                ax_right.imshow(overlay, origin='lower', aspect=aspect)

    ax_right.set_title(f"{plane} slice {slice_index} — Filled TotalSeg Segmentation")
    ax_right.axis("off")

    legend_handles_right = [
        plt.Line2D([0], [0], color=overlay_colors[n][:3], lw=4, label=n)
        for n in ["mandible_totalseg_filled"]
    ]
    leg_right = ax_right.legend(
        handles=legend_handles_right,
        loc='lower right',
        fontsize=8,
        frameon=True,
        facecolor='black',
        edgecolor='white',
        labelcolor='white',
        title="Segments",
        title_fontsize=9
    )

    plt.setp(leg_right.get_texts(), color='white')
    leg_right.get_title().set_color('white')

    plt.subplots_adjust(wspace=0.02)
    clear_output(wait=True)
    display(fig)
    plt.close(fig)


plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=X.shape[2] // 2,
    min=0,
    max=X.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)

def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = X.shape[2] - 1
        slice_slider.value = X.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = X.shape[1] - 1
        slice_slider.value = X.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = X.shape[0] - 1
        slice_slider.value = X.shape[0] // 2

plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(show_slice, slice_index=slice_slider, plane=plane_dropdown, alpha=alpha_slider)


interactive(children=(IntSlider(value=124, description='Slice:', layout=Layout(width='80%'), max=248), Dropdow…

<function __main__.show_slice(slice_index, plane, alpha=0.35)>

In [46]:
import nibabel as nib
import numpy as np
import surface_distance
from surface_distance import metrics
from skimage.metrics import structural_similarity as ssim

manual_path = r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0xZr07hZr1\comparison\mandible_manual.nii.gz"
totalseg_path = r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0xZr07hZr1\comparison\mandible_totalseg_filled.nii.gz"

img_manual = nib.load(manual_path)
img_totalseg = nib.load(totalseg_path)

mask_manual = img_manual.get_fdata() > 0
mask_totalseg = img_totalseg.get_fdata() > 0

mask_manual_float = mask_manual.astype(np.float32)
mask_totalseg_float = mask_totalseg.astype(np.float32)

spacing_mm = img_manual.header.get_zooms()[:3]

surface_distances = surface_distance.compute_surface_distances(
    mask_manual, mask_totalseg, spacing_mm
)

asd = surface_distance.compute_average_surface_distance(surface_distances)
hausdorff_100 = surface_distance.compute_robust_hausdorff(surface_distances, 100)
hausdorff_95 = surface_distance.compute_robust_hausdorff(surface_distances, 95)
surface_overlap_1mm = surface_distance.compute_surface_overlap_at_tolerance(
    surface_distances, tolerance_mm=1
)
surface_dice_1mm = surface_distance.compute_surface_dice_at_tolerance(
    surface_distances, tolerance_mm=1
)
volumetric_dice = surface_distance.compute_dice_coefficient(mask_manual, mask_totalseg)

mae = np.mean(np.abs(mask_manual_float - mask_totalseg_float))

ssim_value = ssim(
    mask_manual_float,
    mask_totalseg_float,
    data_range=1.0,
    win_size=None,  
    channel_axis=None   
)

print("\nSurface Distance Metrics")
print(f"Voxel spacing (mm): {tuple(float(v) for v in spacing_mm)}")
print(f"Average Surface Distance (GT→Pred, Pred→GT): "
      f"({float(asd[0]):.3f}, {float(asd[1]):.3f}) mm")
print(f"Robust Hausdorff (100%): {float(hausdorff_100):.3f} mm")
print(f"Robust Hausdorff (95%): {float(hausdorff_95):.3f} mm")
print(f"Surface Overlap @1mm (GT, Pred): "
      f"({float(surface_overlap_1mm[0]):.3f}, {float(surface_overlap_1mm[1]):.3f})")
print(f"Surface Dice @1mm: {float(surface_dice_1mm):.3f}")


Surface Distance Metrics
Voxel spacing (mm): (0.8788999915122986, 0.8788999915122986, 1.25)
Average Surface Distance (GT→Pred, Pred→GT): (0.460, 0.441) mm
Robust Hausdorff (100%): 6.726 mm
Robust Hausdorff (95%): 2.329 mm
Surface Overlap @1mm (GT, Pred): (0.901, 0.919)
Surface Dice @1mm: 0.910
